In [1]:
import pandas as pd
import os
from functools import reduce
import json


# Directory

In [2]:
### 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()

### downloads 상대경로를 절대경로로 변환
relative_path_downloads = r'..\\..\\..\\Downloads'
download_path = os.path.join(current_directory, relative_path_downloads)

### "preprocessed" 상대경로를 절대경로로 변환
preprocessed_path = os.path.join(download_path, 'preprocessed')

### Check if the "preprocessed" folder exists
if os.path.exists(preprocessed_path):
    print(f"The 'preprocessed' folder already exists at: {preprocessed_path}")
else:
    # "preprocessed" 폴더 생성
    os.makedirs(preprocessed_path)

### ContentsList 상대경로를 절대경로로 변환
relative_path_contentslist = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\001_ContentsList.xlsx'
contentslist_path = os.path.join(current_directory, relative_path_contentslist)

### DailyRawdata
path_dailyrawdata = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\01_DailyRawdata'


### 01_콘텐츠성과측정 상대경로를 절대경로로 변환
relative_path_upper_path = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정'
upper_path = os.path.join(current_directory, relative_path_upper_path)


### 02_ConvertedData 상대경로를 절대경로로 변환
relative_path_convert_path = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData'
convert_path = os.path.join(current_directory, relative_path_convert_path)

### 02_ConvertedData\new_id 상대경로를 절대경로로 변환
relative_path_newid_path = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData\original+new_id'
newid_path = os.path.join(current_directory, relative_path_newid_path)


### new_id 파일 경로 설정
mapping_file_path = os.path.join(relative_path_convert_path, 'id_mapping.json')


### 03_CombinedData 상대경로를 절대경로로 변환
relative_path_combine_path = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\03_CombinedData'
combine_path = os.path.join(current_directory, relative_path_combine_path)


### 04_RequestedData 상대경로를 절대경로로 변환
relative_path_request_path = r'..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\04_RequestedData'
request_path = os.path.join(current_directory, relative_path_request_path)




The 'preprocessed' folder already exists at: c:\Users\ch.kang\Documents\GitHub\funnel_test\GA\..\\..\\..\\Downloads\preprocessed


In [3]:
# 폴더 내의 파일 목록 가져오기
file_list = os.listdir(path_dailyrawdata)

# 'ga'로 시작하는 전체 파일 필터링
ga_files = [file for file in file_list if file.startswith('ga')]

# 'ga'로 시작하는 날짜별 파일 필터링: 주석에 유의하자
# 날짜 범위
# start_date = datetime(2024, 3, 22)
# end_date = datetime(2024, 3, 24)
#ga_files = [file for file in file_list if file.startswith('ga') and start_date <= datetime.strptime(file.split('_')[1].split('.')[0], '%Y%m%d') <= end_date]

# 모든 '아이디' 값들을 저장할 빈 DataFrame 생성
all_ids = pd.DataFrame()

for file in ga_files:
    file_path = os.path.join(path_dailyrawdata, file)
    temp_df = pd.read_excel(file_path, dtype=str)
    
    # 현재 파일에서 '아이디' 열만 추출하여 all_ids에 추가
    current_ids = temp_df[['아이디']].copy()  # '아이디' 열만 선택
    all_ids = pd.concat([all_ids, current_ids], ignore_index=True)

# 중복 제거
all_ids_unique = all_ids.drop_duplicates().reset_index(drop=True)
all_ids_unique['new_id'] = ['g-' + f'{i:06d}' for i in range(1, len(all_ids_unique) + 1)]
all_ids_unique.rename(columns={'아이디': 'pseudo_id'}, inplace=True)
# 매핑 정보 파일 경로
all_ids_unique.to_csv(mapping_file_path, index=False)
# 결과 확인
print(all_ids_unique)

# 새로운 '아이디' 업데이트 함수
def update_id_mapping(existing_mapping_df, new_ids):
    if not existing_mapping_df.empty:
        max_new_id = max(existing_mapping_df['new_id'].apply(lambda x: int(x.split('-')[1])))+1
    else:
        max_new_id = 1
    new_rows = []
    for new_id in new_ids:
        if new_id not in existing_mapping_df['pseudo_id'].values:
            new_row = {'pseudo_id': new_id, 'new_id': f'g-{max_new_id:06d}'}
            new_rows.append(new_row)
            max_new_id += 1
    if new_rows:
        updated_df = pd.concat([existing_mapping_df, pd.DataFrame(new_rows)], ignore_index=True)
    else:
        updated_df = existing_mapping_df
    return updated_df

# 매핑 정보 저장 함수
def save_id_mapping(mapping_df, mapping_file_path):
    mapping_df.to_csv(mapping_file_path, index=False)




# Convert GA Data

In [4]:
for file in ga_files:
    # 파일 이름과 확장자 분리 및 "_pivot" 접미사 추가
    file_name, file_ext = os.path.splitext(file)
    file_name_with_suffix = file_name + "_newID"
    new_file_name = file_name_with_suffix + file_ext

    # 최종 파일 경로 생성
    file_path_convert = os.path.join(newid_path, new_file_name)

    # 만약 변환된 파일이 이미 존재한다면, 다음 파일로 넘어감
    if os.path.exists(file_path_convert):
        print(f"File already exists: {file_path_convert}")
        continue  # 다음 파일로 넘어감

    file_path = os.path.join(path_dailyrawdata, file)  # 파일의 전체 경로 생성
    temp_df = pd.read_excel(file_path, dtype=str) # 파일을 데이터프레임으로 읽기

    # # new_id 업데이트
    # ids = set(temp_df['아이디'].unique())  # 고유 '아이디' 추출
    # id_mapping_df = update_id_mapping(id_mapping_df, list(ids))  # 집합을 리스트로 변환하여 함수에 전달
    # save_id_mapping(id_mapping_df, mapping_file_path)
    # 데이터프레임 로드

    # 고유 아이디 매핑 파일 로드
    with open(mapping_file_path, 'r') as f:
        id_mapping = json.load(f)

    # UniqueIDGenerator 클래스
    class UniqueIDGenerator:
        def __init__(self, id_mapping):
            self.id_mapping = id_mapping

        def get_unique_id(self, username):
            return self.id_mapping.get(username, None)

    # UniqueIDGenerator 인스턴스 생성
    generator = UniqueIDGenerator(id_mapping)

    # 고유 아이디 생성 및 데이터프레임에 추가
    temp_df['고유아이디'] = temp_df['아이디'].apply(generator.get_unique_id)
    
    # 출력, 저장
    temp_df.to_excel(file_path_convert)




File already exists: c:\Users\ch.kang\Documents\GitHub\funnel_test\GA\..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData\original+new_id\ga_20240115_newID.xlsx
File already exists: c:\Users\ch.kang\Documents\GitHub\funnel_test\GA\..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData\original+new_id\ga_20240116_newID.xlsx
File already exists: c:\Users\ch.kang\Documents\GitHub\funnel_test\GA\..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData\original+new_id\ga_20240117_newID.xlsx
File already exists: c:\Users\ch.kang\Documents\GitHub\funnel_test\GA\..\\..\\..\\..\\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData\original+new_id\ga_20240118_newID.xlsx
File already exists: c:\Users\ch.kang\Documents\GitHub\funnel_test\GA\..\\..\\..\\..\\OneDrive -